<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/Pytorch012.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AE（包含卷积）

In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder()
if torch.cuda.is_available():
    model.cuda()
#model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    total_loss = 0
    for data in dataloader:
        img, _ = data
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, total_loss))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './conv_autoencoder.pth')

100%|██████████| 9912422/9912422 [00:00<00:00, 77992674.99it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 67522683.29it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 28234927.58it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14132439.74it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



epoch [1/100], loss:186.2933
epoch [2/100], loss:106.8397
epoch [3/100], loss:82.3638
epoch [4/100], loss:73.6374
epoch [5/100], loss:69.2785
epoch [6/100], loss:66.3837
epoch [7/100], loss:64.1606
epoch [8/100], loss:62.5021
epoch [9/100], loss:61.2466
epoch [10/100], loss:60.2273
epoch [11/100], loss:59.3377
epoch [12/100], loss:58.7013
epoch [13/100], loss:58.1021
epoch [14/100], loss:57.5438
epoch [15/100], loss:56.9899
epoch [16/100], loss:56.6212
epoch [17/100], loss:56.2619
epoch [18/100], loss:55.9111
epoch [19/100], loss:55.6041
epoch [20/100], loss:55.2879
epoch [21/100], loss:54.9531
epoch [22/100], loss:54.6254
epoch [23/100], loss:54.3218
epoch [24/100], loss:53.9432
epoch [25/100], loss:53.4427
epoch [26/100], loss:52.9927
epoch [27/100], loss:52.6488
epoch [28/100], loss:52.2780
epoch [29/100], loss:51.9335
epoch [30/100], loss:51.5701
epoch [31/100], loss:51.2124
epoch [32/100], loss:50.9080
epoch [33/100], loss:50.5378
epoch [34/100], loss:50.2258
epoch [35/100], loss: